In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from tqdm import tqdm


from model import AlexNet, LeNet

In [5]:
## Test learning to reweight model
# from main import Reweighting


# torch.backends.cudnn.enabled = False
# torch.manual_seed(1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device = ", device)

class Reweighting():
    def __init__(self, network, hyperparameters, criterion, criterion_mean, optimizer, train_loader, valid_loader, test_loader):
        self.network = network.requires_grad_(requires_grad=True)
        self.hyperparameters = hyperparameters
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader
        self.criterion_mean = criterion_mean
        self.gradient_network = None

    def train(self):
        # Train the network
        for epoch in range(self.hyperparameters['n_epochs']):
            self.network.train()
            for batch_idx, (data, target) in enumerate(self.train_loader):
                data = data.to(device)
                target = target.to(device)

                self.optimizer.zero_grad()
                output = self.network(data)
                # print(output)
                loss = self.criterion_mean(output, target)
                # print(loss)
                loss.backward()
                self.optimizer.step()
                if batch_idx % self.hyperparameters['log_interval'] == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(self.train_loader.dataset),
                        100. * batch_idx / len(self.train_loader), loss.item()))


    def test(self):
        self.network.eval()

        acc = []
        for itr,(test_img, test_label) in tqdm(enumerate(self.test_loader)):
            prediction = self.network(test_img.to(device)).detach().cpu().numpy()
            # print(prediction)
            prediction = np.argmax(prediction, axis=1)
            tmp = (prediction == test_label.detach().numpy())
            # print(prediction)
            # print(test_label)
            acc.append(tmp)

        accuracy = np.concatenate(acc).mean()
        return np.round(accuracy*100,2)


device =  cuda:0


In [3]:
train_folder = '../dataset-ninja/train_unbiased'
test_folder = '../dataset-ninja/test_unbiased'
validate_folder = '../dataset-ninja/validate_unbiased'

class_weights = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Example weights for each class

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[90.2867748375, 88.24014045, 94.9276596], std=[46.843379357493816, 46.59302413225906, 47.86539694070785]),  # Normalize images
])

train_dataset = datasets.ImageFolder(train_folder, transform=transform)
weights = [class_weights[label] for label in train_dataset.targets]
train_sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

test_dataset = datasets.ImageFolder(test_folder, transform=transform)
weights = [class_weights[label] for label in test_dataset.targets]
test_sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

validate_dataset = datasets.ImageFolder(validate_folder, transform=transform)
weights = [class_weights[label] for label in validate_dataset.targets]
valid_sampler = WeightedRandomSampler(weights, len(weights), replacement=True)


# number of epoch and log interval reduced for testing
hyperparameters = {
    'n_epochs' : 10,
    'batch_size' : 1024,
    'learning_rate' : 1e-3,
    'momentum' : 0.5,
    'log_interval' : 2
}

network = AlexNet()

criterion = nn.CrossEntropyLoss(reduction='none')
criterion_mean = nn.CrossEntropyLoss(reduction='mean')

optimizer = optim.SGD(network.params(),
                        lr=hyperparameters['learning_rate'],
                        momentum=hyperparameters['momentum'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], sampler=test_sampler)
valid_loader = DataLoader(validate_dataset, batch_size=hyperparameters['batch_size'], sampler=valid_sampler)

our_model = Reweighting(network, hyperparameters, criterion, criterion_mean, optimizer, train_loader, valid_loader, test_loader)

start_accuracy = our_model.test()
print("Starting accuracy = ", start_accuracy)

our_model.train()

end_accuracy = our_model.test()
print("Ending accuracy = ", end_accuracy)



24it [00:36,  1.52s/it]


Starting accuracy =  1.27
Train Epoch: 0 [0/190879 (0%)]	Loss: 2.083866
Train Epoch: 0 [2048/190879 (1%)]	Loss: 2.083385
Train Epoch: 0 [4096/190879 (2%)]	Loss: 2.081394
Train Epoch: 0 [6144/190879 (3%)]	Loss: 2.078940
Train Epoch: 0 [8192/190879 (4%)]	Loss: 2.076996
Train Epoch: 0 [10240/190879 (5%)]	Loss: 2.075287
Train Epoch: 0 [12288/190879 (6%)]	Loss: 2.072867
Train Epoch: 0 [14336/190879 (7%)]	Loss: 2.070789
Train Epoch: 0 [16384/190879 (9%)]	Loss: 2.068324
Train Epoch: 0 [18432/190879 (10%)]	Loss: 2.067001
Train Epoch: 0 [20480/190879 (11%)]	Loss: 2.064949
Train Epoch: 0 [22528/190879 (12%)]	Loss: 2.062800
Train Epoch: 0 [24576/190879 (13%)]	Loss: 2.060368
Train Epoch: 0 [26624/190879 (14%)]	Loss: 2.058346
Train Epoch: 0 [28672/190879 (15%)]	Loss: 2.056059
Train Epoch: 0 [30720/190879 (16%)]	Loss: 2.054125
Train Epoch: 0 [32768/190879 (17%)]	Loss: 2.051368
Train Epoch: 0 [34816/190879 (18%)]	Loss: 2.048727
Train Epoch: 0 [36864/190879 (19%)]	Loss: 2.047882
Train Epoch: 0 [38912/

KeyboardInterrupt: 

In [7]:
our_model.network.eval()

acc = []
for itr,(test_img, test_label) in tqdm(enumerate(our_model.test_loader)):
    prediction = our_model.network(test_img.to(device)).detach().cpu().numpy()
    # print(prediction)
    prediction = np.argmax(prediction, axis=1)
    tmp = (prediction == test_label.detach().numpy())
    print(prediction)
    print(test_label)
    acc.append(tmp)

accuracy = np.concatenate(acc).mean()
end_accuracy = np.round(accuracy*100,2)

print("Ending accuracy = ", end_accuracy)

1it [00:01,  1.42s/it]

[2 2 2 ... 2 2 2]
tensor([7, 2, 2,  ..., 2, 2, 7])


2it [00:02,  1.32s/it]

[2 2 2 ... 2 2 2]
tensor([7, 2, 2,  ..., 7, 7, 2])


3it [00:03,  1.27s/it]

...

[2 2 2 ... 2 2 2]
tensor([7, 2, 2,  ..., 2, 2, 2])


23it [00:27,  1.15s/it]

[2 2 2 ... 2 2 2]
tensor([2, 2, 7,  ..., 6, 2, 4])


24it [00:27,  1.16s/it]

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2]
tensor([2, 2, 2, 6, 7, 2, 7, 6, 2, 2, 2, 2, 7, 1, 7, 2, 4, 2, 2, 2, 7, 7, 2, 7,
        2, 7, 7, 4, 2, 7, 2, 7, 2, 2, 2, 4, 7, 7, 2, 6, 6, 7, 2, 2, 7, 2, 2, 7,
        6, 2, 2, 2, 7, 7, 7, 7, 7, 1, 6, 2, 2, 2, 2, 4, 2, 7, 2, 7, 7, 7, 2, 2,
        2, 2, 1, 2, 2, 2, 2, 7, 2, 2, 6, 2, 2, 2, 2, 2, 7, 2, 7, 2, 2, 7, 2, 2,
        2, 2, 2, 7, 2, 2, 2, 7, 4, 7, 2, 7, 2, 7, 7, 2

In [15]:
## Test control model
# from main import Reweighting


# torch.backends.cudnn.enabled = False
# torch.manual_seed(1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device = ", device)

class NoReweighting():
    def __init__(self, network, hyperparameters, criterion, optimizer, train_loader, test_loader):
        self.network = network
        self.hyperparameters = hyperparameters
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.train_loader
        self.test_loader = test_loader

    def train(self):
        # Train the network
        for epoch in range(hyperparameters['n_epochs']):
            self.network.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                data = data.to(device)
                target = target.to(device)

                self.optimizer.zero_grad()
                output = self.network(data)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()
                # if batch_idx % hyperparameters['log_interval'] == 0:
                #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #         epoch, batch_idx * len(data), len(train_loader.dataset),
                #         100. * batch_idx / len(train_loader), loss.item()))


    def test(self):
        test_loss = 0
        correct = 0
        for batch_idx, (data, target) in enumerate(test_loader):
            self.network.eval()
            output = self.network(data.to(device)).cpu()
            test_loss += self.criterion(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()

        # print('\nTest set: \nAvg. loss: {:.4f} \nAccuracy: {}/{} ({:.2f}%)\n'.format(
        #     test_loss, correct, len(test_loader.dataset),
        #     100.0 * correct / len(test_loader.dataset)))
        
        return (100.0 * correct / len(test_loader.dataset)).item()

# number of epoch and log interval reduced for testing
hyperparameters = {
    'n_epochs' : 2,
    'batch_size_train' : 100,
    'batch_size_valid' : 10,
    'batch_size_test' : 1000,
    'learning_rate' : 1e-3,
    'momentum' : 0.5,
    'log_interval' : 1
}

network = LeNet()

criterion = nn.CrossEntropyLoss(reduction='none')
criterion_mean = nn.CrossEntropyLoss(reduction='mean')

optimizer = optim.SGD(network.params(),
                        lr=hyperparameters['learning_rate'],
                        momentum=hyperparameters['momentum'])
# Load the data
data_loader = MNISTDataLoader(validation_ratio=0.05,
                            batch_size_train=hyperparameters['batch_size_train'],
                            batch_size_valid=hyperparameters['batch_size_valid'],
                            batch_size_test=hyperparameters['batch_size_test'])

# 10% representation by 9's class in training data
desired_sample_distribution = [100, 100, 100, 100, 100, 100, 100, 100, 100, 10]
data_loader.sample_bias(desired_sample_distribution, dataset="train")


train_loader = data_loader.train_dataloader
valid_loader = data_loader.valid_dataloader
test_loader = data_loader.test_dataloader


our_model = NoReweighting(network, hyperparameters, criterion_mean, optimizer, train_loader, test_loader)

start_accuracy = our_model.test()
print("Starting accuracy = ", start_accuracy)

our_model.train()

end_accuracy = our_model.test()
print("Ending accuracy = ", end_accuracy)


device =  cuda:0
Starting accuracy =  9.569999694824219
Ending accuracy =  18.65999984741211
